In [13]:
import pandas as pd
import numpy as np 
import json
import os
import re
import spacy 
spacy.prefer_gpu(1)
nlp = spacy.load("en_core_web_trf")

In [14]:
read = pd.read_json("raw/out_label/summaries_outx.json", lines = True)
df = read[read.answer == "accept"]
df.head()

,text,article_id,_input_hash,_task_hash,_is_binary,spans,tokens,_view_id,relations,answer,_timestamp
0,Ukraine war: Russian tactics on eastern front ...,0,1192826571,1725000051,False,"[{'start': 0, 'end': 7, 'token_start': 0, 'tok...","[{'text': 'Ukraine', 'start': 0, 'end': 7, 'id...",relations,"[{'head': 3, 'child': 0, 'head_span': {'start'...",accept,1666853824
2,Lee Jae-yong: Samsung appoints Lee Jae-yong to...,2,-386898449,495824270,False,"[{'start': 0, 'end': 12, 'token_start': 0, 'to...","[{'text': 'Lee', 'start': 0, 'end': 3, 'id': 0...",relations,"[{'head': 31, 'child': 18, 'head_span': {'star...",accept,1666854104
3,New Zealand Instagram couple 'relieved' after ...,3,94784817,-1047716132,False,"[{'start': 0, 'end': 28, 'token_start': 0, 'to...","[{'text': 'New', 'start': 0, 'end': 3, 'id': 0...",relations,"[{'head': 3, 'child': 9, 'head_span': {'start'...",accept,1666854141
4,Iran protests: Police fire on Mahsa Amini mour...,4,-1204324385,1067222802,False,"[{'start': 0, 'end': 4, 'token_start': 0, 'tok...","[{'text': 'Iran', 'start': 0, 'end': 4, 'id': ...",relations,"[{'head': 3, 'child': 8, 'head_span': {'start'...",accept,1666854246
5,Germany plans to legalise recreational cannabi...,5,1523244431,980527859,False,"[{'start': 0, 'end': 7, 'token_start': 0, 'tok...","[{'text': 'Germany', 'start': 0, 'end': 7, 'id...",relations,"[{'head': 0, 'child': 5, 'head_span': {'start'...",accept,1666854286


In [15]:
def annot_to_sent(df):
    new = []
    for index, row in df.iterrows():
        doc = nlp(row.text)
        splits = []
        cur_sent = ""
        last_end = True
        start_idx = 0
        last_idx = 0

        for sent in doc.sents:
            sent = sent.text
            last_idx = start_idx + len(sent)
            splits.append([start_idx,last_idx, sent])
            start_idx += len(cur_sent + sent) + 1

            relations = row.relations.copy()
            for i in range(len(splits)):
                sub = splits.pop(0)

                start = sub[0]
                lim = sub[1]
                text = sub[2]

                for idx, rel in enumerate(relations):
                    if rel["head_span"]["end"] <= lim and rel["child_span"]["end"] <= lim and rel["head_span"]["start"] >= start and rel["child_span"]["start"] >= start:
                        subj = text[rel["head_span"]["start"]-start:rel["head_span"]["end"]-start]
                        obj = text[rel["child_span"]["start"]-start:rel["child_span"]["end"]-start]
                        new.append([text, f"<triplet> {subj} <subj> {obj} <obj> {rel['label']}"])
                        relations[idx] == [9999,9999,"found already"]

    last_txt = ""
    last_rel = ""
    alligned = []
    for sent in new:
        if sent[0] == last_txt:
            last_rel = last_rel.lstrip() + " " + sent[1]
        elif sent[0] != last_txt:
            alligned.append([last_txt, last_rel])
            last_txt = sent[0]
            last_rel = sent[1]
    if alligned[-1][0] == last_txt:
        alligned[-1][1] == alligned[-1][1] + " " + sent[1]
    else:
        alligned.append([last_txt, last_rel])
    alligned.pop(0)

    df = pd.DataFrame(alligned, columns = ["text","label"])
    return df

In [16]:
df = annot_to_sent(df)

In [17]:
read = pd.read_json("raw/out_label/counter_annotate.json", lines = True)
df_2 = read[read.answer == "accept"]
df_2.head()

,text,article_id,_input_hash,_task_hash,_is_binary,spans,tokens,_view_id,relations,answer,_timestamp
0,"Sensible America finally spoke, but ‘Trumpty D...",3159,2108892512,770305806,False,"[{'start': 68, 'end': 89, 'token_start': 15, '...","[{'text': 'Sensible', 'start': 0, 'end': 8, 'i...",relations,"[{'head': 17, 'child': 22, 'head_span': {'star...",accept,1672333765
1,How Was Russia Able to Launch Russia's Biggest...,4317,-1406124216,-723512353,False,"[{'start': 8, 'end': 14, 'token_start': 2, 'to...","[{'text': 'How', 'start': 0, 'end': 3, 'id': 0...",relations,"[{'head': 18, 'child': 21, 'head_span': {'star...",accept,1672333840
2,World leaders discuss climate change solutions...,2396,-2130737925,1317932275,False,"[{'start': 0, 'end': 13, 'token_start': 0, 'to...","[{'text': 'World', 'start': 0, 'end': 5, 'id':...",relations,"[{'head': 1, 'child': 7, 'head_span': {'start'...",accept,1672333863
3,"In Colombia, Drilling Pays the Bills. Colombia...",3831,-534305385,-1860660404,False,"[{'start': 38, 'end': 54, 'token_start': 8, 't...","[{'text': 'In', 'start': 0, 'end': 2, 'id': 0,...",relations,"[{'head': 9, 'child': 13, 'head_span': {'start...",accept,1672333943
4,Photo: Mohamed Salah shares image of Mohamed S...,1188,343647075,791917291,False,"[{'start': 7, 'end': 20, 'token_start': 2, 'to...","[{'text': 'Photo', 'start': 0, 'end': 5, 'id':...",relations,"[{'head': 51, 'child': 46, 'head_span': {'star...",accept,1672049094


In [18]:
df_2 = annot_to_sent(df_2)

In [19]:
df.shape, df_2.shape

((1720, 2), (119, 2))

In [43]:
con = pd.concat([df, df_2])
con = con[~con.duplicated(["text","label"])]
con = con.reset_index()
con.shape

(1835, 3)

In [44]:
con["label"] = con.label.apply(lambda x: x.replace(
    "ProvideAid", "Provide Aid").replace(
    "EngageInDiplomaticCooperation", "Engage In Diplomatic Cooperation").replace(
    "MakePublicStatement", "Make Public Statement").replace(
    "ExpressIntendToCooperate","Express Intend to Cooperate").replace(
    "EngageInMaterialCooperation","Engage In Material Cooperation").replace(
    "ExhibitMilitaryPosture","Exhibit Military Posture").replace(
    "ReduceRelations", "Reduce Relations").replace(
    "ExpressIntentToCooperate", "Express Intend to Cooperate").replace(
    "EngageInUnconventialMassViolence", "Engage in unconventional mass violence"))

In [45]:

#extract the categorized relation for stratified split on cameo codes
relation = []
len_cat = []
for row in con.iterrows():
    rel_iter = row[1]["label"] + " <triplet>"
    all_rels = re.findall("(?<=<obj> ).*?(?= <triplet>)", rel_iter)
    relation.append(all_rels)
con["relations"] = relation
con["rel_len"] = con.relations.apply(lambda x: len(x))
#fix "Protest" and "Fight" being found in entities

In [46]:
con.explode("relations").groupby("relations").text.count(), con.rel_len.sum()

(relations
 Appeal                                      96
 Assault                                    104
 Coerce                                     128
 Consult                                    289
 Demand                                      50
 Disapprove                                 274
 Engage In Diplomatic Cooperation           179
 Engage In Material Cooperation             124
 Engage in unconventional mass violence       5
 Exhibit Military Posture                    53
 Express Intend to Cooperate                 46
 Fight                                      211
 Investigate                                 83
 Make Public Statement                     1098
 Protest                                     60
 Provide Aid                                 88
 Reduce Relations                           151
 Reject                                      41
 Threaten                                    45
 Yield                                       91
 Name: text, dtype: int64,
 3

In [47]:
con["len_c"] = con.relations.apply(lambda x: len(x))
con["MPS"] = con.relations.apply(lambda x: x.count("Make Public Statement"))
con["MPS%"] = con["MPS"] / con["len_c"]
sub = con[(con["MPS%"] > 0.5) & (con["MPS"]>2)].sort_values(["MPS%","MPS","len_c"], ascending = False)
sub.describe()

,index,rel_len,len_c,MPS,MPS%
count,66.000000,66.000000,66.000000,66.000000,66.000000
mean,893.045455,4.575758,4.575758,3.318182,0.746068
std,515.881245,0.993217,0.993217,0.636314,0.151749
min,52.000000,3.000000,3.000000,3.000000,0.571429
25%,486.750000,4.000000,4.000000,3.000000,0.600000
50%,971.000000,5.000000,5.000000,3.000000,0.750000
75%,1346.500000,5.000000,5.000000,3.000000,0.800000
max,1671.000000,7.000000,7.000000,6.000000,1.000000


In [48]:
sub.explode("relations").groupby("relations").text.count(), con.rel_len.sum()

(relations
 Appeal                                3
 Assault                               4
 Coerce                                8
 Consult                              13
 Demand                                2
 Disapprove                            7
 Engage In Diplomatic Cooperation      2
 Engage In Material Cooperation        3
 Exhibit Military Posture              4
 Express Intend to Cooperate           1
 Fight                                20
 Investigate                           1
 Make Public Statement               219
 Protest                               1
 Provide Aid                           2
 Reduce Relations                      7
 Reject                                2
 Threaten                              2
 Yield                                 1
 Name: text, dtype: int64,
 3216)

In [49]:
sub.MPS.sum()
#removing these 66 rows, with hardly any value

219

In [50]:
sub.shape

(66, 8)

In [51]:
con = con.drop(index = sub.index)
con.shape

(1769, 8)

In [52]:
con[["text","label"]].to_csv("annotated/sent_full.csv")

# Data from ConfliBERT (not used, just checked)

In [2]:
input = "conflibert,hu_et_al.2022/ner_train.txt"

In [3]:
parsed = open(input, encoding = "utf-8")
parsedfile = parsed.readlines()

In [4]:
parsedlines = []
cur = ""
ents = []
ent = ""
for line in parsedfile:
    if line == "\n":
        #cur = re.sub(" \W","", cur.lstrip())
        parsedlines.append([cur.lstrip(), ents])
        ents = []
        cur = ""
    else:
        split = line.replace("\n","").split()
        if split[0] != "," and split[0] != ":" and split[0] != "." and split[0] != "'" and split[0] != "’":
            cur += " " + split[0]
        else:
            cur += split[0]
        if split[1] != "O":
            ent += " " + split[0]
        elif split[1] == "O" and ent != "":
            ents.append(ent.lstrip())
            ent = ""

In [5]:
len(parsedlines)

1153

In [6]:
df = pd.DataFrame(parsedlines, columns = ["text","ents"])

In [7]:
df["ent_len"] = df.ents.apply(lambda x: len(x))

In [8]:
df = df[df.ent_len == 2]

In [9]:
df["start"] = df.text.apply(lambda x: " ".join(x.split()[:5]))

In [10]:
df_2 = pd.read_csv("conflibert,hu_et_al.2022/rcc_train.tsv", sep = "\t", header = None)
df_2.columns =  ["text","label"]
df_2["start"] = df_2.text.apply(lambda x: " ".join(x.split()[:5]))

In [11]:
merge = pd.merge(df, df_2, on = "start", how = "inner")

In [12]:
merge = merge[["text_y","ents","label"]]

In [36]:
merge.text_y.to_list()[125]

'Congress will fully support the Philippine Plan of Action to End Violence Against Children (PPAEVAC), Davao City Rep. Karlo Nograles said yesterday.'

In [37]:
merge.ents.to_list()[125]

['Congress',
 'the Philippine Plan of Action to End Violence Against Children ( PPAEVAC )']